# Analyze eBPF Programs

This notebook compiles, analyzes, and generates the report for eBPF programs. 

The raw report is saved in the `output/report` directory.

In [1]:
%reload_ext autoreload
%autoreload 2

The following cell compiles the BCC tools. 

In [2]:
from utils import BCC_TOOLS_PATH

!make -C {BCC_TOOLS_PATH} -j $(nproc)

make: Entering directory '/Users/szhong/Code/DepSurf/software/bcc/libbpf-tools'
make: Nothing to be done for 'all'.
make: Leaving directory '/Users/szhong/Code/DepSurf/software/bcc/libbpf-tools'


The following cell compiles the eBPF program for Tracee.

In [3]:
from utils import TRACEE_PATH

!make -C {TRACEE_PATH} bpf

make: Nothing to be done for 'bpf'.


The following cell analyzes and generates the report for eBPF programs.

In [2]:
from depsurf import BPFProgram, VersionGroup, DepReport, DepKind
from utils import TRACEE_PATH, BCC_OBJ_PATH, OUTPUT_PATH, save_pkl

REPORT_PATH = OUTPUT_PATH / "report"
REPORT_PATH.mkdir(parents=True, exist_ok=True)
GROUPS = [VersionGroup.REGULAR, VersionGroup.ARCH]
PROG_PATHS = sorted(BCC_OBJ_PATH.glob("*.bpf.o")) + [TRACEE_PATH / "dist" / "tracee.bpf.o"]


REPORT_KINDS = [
    DepKind.FUNC,
    DepKind.TRACEPOINT,
    DepKind.LSM,
    DepKind.FIELD,
    DepKind.STRUCT,
    DepKind.SYSCALL,
]

results = {}
for path in PROG_PATHS:
    bpf = BPFProgram(path)
    result_path = REPORT_PATH / f"{bpf.name}.log"
    with open(result_path, "w") as f:
        prog_report = {}
        for dep in bpf.deps:
            if dep.kind not in REPORT_KINDS:
                print(f"{dep}: Skipped", file=f)
                continue
            report = DepReport.from_groups(dep, GROUPS)
            report.print(file=f)
            prog_report[dep] = report
        print(f"Report saved to {result_path}", flush=True)
    results[bpf.name] = prog_report


save_pkl(results, "bcc")

[         dump.py:161] INFO: Normalizing types from /Users/szhong/Code/DepSurf/software/bcc/libbpf-tools/.output/bashreadline.bpf.min.btf.json


Report saved to /Users/szhong/Code/DepSurf/output/report/bashreadline.log


[         dump.py:161] INFO: Normalizing types from /Users/szhong/Code/DepSurf/software/bcc/libbpf-tools/.output/bindsnoop.bpf.min.btf.json
[       groups.py:45 ] INFO: Loading funcs from /Users/szhong/Code/DepSurf/data/dataset/func_groups/4.4.0-21-generic-amd64.jsonl
[       groups.py:45 ] INFO: Loading funcs from /Users/szhong/Code/DepSurf/data/dataset/func_groups/4.8.0-22-generic-amd64.jsonl
[       groups.py:45 ] INFO: Loading funcs from /Users/szhong/Code/DepSurf/data/dataset/func_groups/4.10.0-19-generic-amd64.jsonl
[       groups.py:45 ] INFO: Loading funcs from /Users/szhong/Code/DepSurf/data/dataset/func_groups/4.13.0-16-generic-amd64.jsonl
[       groups.py:45 ] INFO: Loading funcs from /Users/szhong/Code/DepSurf/data/dataset/func_groups/4.15.0-20-generic-amd64.jsonl
[       groups.py:45 ] INFO: Loading funcs from /Users/szhong/Code/DepSurf/data/dataset/func_groups/4.18.0-10-generic-amd64.jsonl
[       groups.py:45 ] INFO: Loading funcs from /Users/szhong/Code/DepSurf/data/da

Report saved to /Users/szhong/Code/DepSurf/output/report/bindsnoop.log


[         dump.py:161] INFO: Normalizing types from /Users/szhong/Code/DepSurf/software/bcc/libbpf-tools/.output/biolatency.bpf.min.btf.json


...

[         dump.py:161] INFO: Normalizing types from /Users/szhong/Code/DepSurf/software/bcc/libbpf-tools/.output/wakeuptime.bpf.min.btf.json


Report saved to /Users/szhong/Code/DepSurf/output/report/wakeuptime.log


[         dump.py:161] INFO: Normalizing types from /Users/szhong/Code/DepSurf/software/tracee/dist/tracee.bpf.min.btf.json


Report saved to /Users/szhong/Code/DepSurf/output/report/tracee.log


[ utils_pickle.py:13 ] INFO: Saved bcc to /Users/szhong/Code/DepSurf/output/bcc.pkl
